# 1 UCI Digits

## 1.1 Data preprocessing

In [1]:
import numpy as np
import pandas as pd
import urllib.request
import os

# Define dataset URL and filenames
BASE_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/"
FILENAMES = {
    "profile": "mfeat-fou",
    "fourier": "mfeat-fac",
    "karhunen": "mfeat-kar",
    "intensity": "mfeat-pix",
    "zernike": "mfeat-zer",
    "morphological": "mfeat-mor"
}

# Define feature dimensions
FEATURE_DIMS = {
    "profile": 216,
    "fourier": 76,
    "karhunen": 64,
    "intensity": 240,
    "zernike": 47,
    "morphological": 6
}

# Create a directory for the dataset
DATA_DIR = "uci_digits"
os.makedirs(DATA_DIR, exist_ok=True)

# Function to download the dataset
def download_data():
    for key, filename in FILENAMES.items():
        file_path = os.path.join(DATA_DIR, filename)
        if not os.path.exists(file_path):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(BASE_URL + filename, file_path)
        else:
            print(f"{filename} already downloaded.")
download_data()

# Load all six feature sets into a dictionary
features = {key: np.loadtxt(os.path.join(DATA_DIR, filename)) for key, filename in FILENAMES.items()}

# Create labels (0-9, 200 samples each)
labels = np.repeat(np.arange(10), 200)

# Convert dataset into a list of (X, y) pairs
dataset_list = [(features[key], labels) for key in FILENAMES.keys()]

# Verify dataset shapes
feature_list = []
for i, (X, y) in enumerate(dataset_list):
    print(f"Feature set {i+1}: X shape {X.shape}, y shape {y.shape}")
    feature_list.append(X)

class_number = 10

mfeat-fou already downloaded.
mfeat-fac already downloaded.
mfeat-kar already downloaded.
mfeat-pix already downloaded.
mfeat-zer already downloaded.
mfeat-mor already downloaded.
Feature set 1: X shape (2000, 76), y shape (2000,)
Feature set 2: X shape (2000, 216), y shape (2000,)
Feature set 3: X shape (2000, 64), y shape (2000,)
Feature set 4: X shape (2000, 240), y shape (2000,)
Feature set 5: X shape (2000, 47), y shape (2000,)
Feature set 6: X shape (2000, 6), y shape (2000,)


## 1.2 Single view graph

In [2]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
from tqdm import tqdm 
from scipy import linalg

def compute_laplacian(S):
    S_sym = (S.T + S) / 2  # Compute symmetric part
    D = np.diag(S_sym.sum(axis=1))  # Compute diagonal matrix D
    L = D - S_sym  # Compute Laplacian matrix
    return L

def update_Q(L, c):
    L = (L + L.T) / 2
    eigenvals, eigenvecs = linalg.eigh(L)
    Q = eigenvecs[:, :c]
    return Q


def update_S(Q, beta):
    n = Q.shape[0]
    S = np.zeros((n, n))
    
    # Calculate pairwise distances (g_ij matrix)
    G = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            G[i, j] = np.sum((Q[i] - Q[j]) ** 2)
    
    # Update each column of S independently
    for j in range(n):
        g_j = G[:, j]  # Get j-th column of G
        
        # Binary search for eta that satisfies sum(s_j) = 1
        left, right = 0, np.max(g_j)/(2*beta)
        
        while right - left > 1e-10:
            eta = (left + right) / 2
            s_j = np.maximum(0, -g_j/(2*beta) + eta)
            sum_s = np.sum(s_j)
            
            if sum_s < 1:
                left = eta
            else:
                right = eta
        
        # Final update for j-th column using found eta
        S[:, j] = np.maximum(0, -g_j/(2*beta) + eta)
    
    S = (S.T + S) / 2
    return S


def make_single_view_graph(single_view_graph_X, class_number, default_beta=1.0):
    
    single_view_graph = []
    
    for i in tqdm(range(len(single_view_graph_X))):
        
        # init
        beta = default_beta
        S = update_S(single_view_graph_X[i], beta)
        L = compute_laplacian(S)
        Q = update_Q(L, class_number)

        for j in range(100):
            S = update_S(Q, beta)
            L = compute_laplacian(S)
            Q = update_Q(L, class_number)

            L_rank = np.linalg.matrix_rank(L)
            # print("L_rank",L_rank, "beta", beta)
            if L_rank == X.shape[0] - class_number:
                tqdm.write(f"{i+1}th graph end at {j}th iteration, L's rank is {L_rank}")
                break
            elif L_rank > X.shape[0] - class_number:
                beta *= 0.9 
            else:
                beta *= 1.1
                
        single_view_graph.append(S)
        
    return single_view_graph


In [3]:
single_view_graph = make_single_view_graph(feature_list, class_number)


 17%|█▋        | 1/6 [04:00<20:04, 240.90s/it]

1th graph end at 12th iteration, L's rank is 1990


 33%|███▎      | 2/6 [06:25<12:17, 184.32s/it]

2th graph end at 7th iteration, L's rank is 1990


 50%|█████     | 3/6 [09:13<08:50, 176.80s/it]

...

4th graph end at 7th iteration, L's rank is 1990


 83%|████████▎ | 5/6 [15:45<03:13, 193.07s/it]

5th graph end at 11th iteration, L's rank is 1990


100%|██████████| 6/6 [19:30<00:00, 195.00s/it]

6th graph end at 11th iteration, L's rank is 1990


## 1.3 Global view graph

In [4]:
def init_W(single_view_graph):
    W = [np.full(single_view_graph[0].shape, 1/len(single_view_graph))] * len(single_view_graph)
    return W

def init_A(single_view_graph, W):
    A = np.sum(single_view_graph, axis=0) * W[0]
    return A

def init_P(A,c):
    L = compute_laplacian(A)
    P = update_Q(L, c)
    return P


def update_A(P, w_list, s_list, gamma=1.0):

    n = P.shape[0]
    A = np.zeros((n, n))
    
    # Compute hj for each column
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = np.sum((P[i] - P[j])**2)  # ||p_i - p_j||^2
    
    # For each column j
    for j in range(n):
        # Compute the weighted sum of similarity matrices for column j
        weighted_sum = np.zeros(n)
        for v in range(len(w_list)):
            weighted_sum += w_list[v][:, j] * s_list[v][:, j]
        
        # Compute b_j = hj - sum_v w_j^(v) * s_j^(v)
        b_j = (gamma/2) *H[:, j] - weighted_sum
        
        # Update each column of S independently
        for j in range(n):
            
            # Binary search for eta that satisfies sum(s_j) = 1
            left, right = 0, np.max(b_j)
            
            while right - left > 1e-10:
                eta = (left + right) / 2
                s_j = np.maximum(0, -b_j + eta)
                sum_s = np.sum(s_j)
                
                if sum_s < 1:
                    left = eta
                else:
                    right = eta
            
            # Final update for j-th column using found eta
            A[:, j] = np.maximum(0, -b_j + eta)
    
    A = (A.T + A) / 2
    return A


def update_P(L, c):
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    Q = eigenvectors[:, :c]
    return Q


def project_simplex(v):
    n = len(v)
    
    # Sort v in descending order
    u = np.sort(v)[::-1]
    
    # Compute the threshold
    cssv = np.cumsum(u) - 1
    ind = np.arange(n) + 1
    cond = u - cssv / ind > 0
    
    if not np.any(cond):  # Handle the case where all elements are negative
        return np.ones(n) / n
    
    rho = ind[cond][-1]
    
    # Compute the Lagrange multiplier
    theta = cssv[rho-1] / rho
    
    # Compute the projection
    x = np.maximum(v - theta, 0)
    
    return x

def compute_W(a, s_list):
    n = a.shape[0]
    nv = len(s_list)  # Number of views
    
    # Initialize weight matrices
    w_list = [np.zeros((n, n)) for _ in range(nv)]
    
    # For each column j
    for j in range(n):
        # Construct Z_j matrix where each column corresponds to a view
        # and each row corresponds to a sample
        Z_j = np.zeros((n, nv))
        for v in range(nv):
            Z_j[:, v] = a[:, j] - s_list[v][:, j]
        
        # Compute Z_j^T * Z_j
        ZTZ = np.dot(Z_j.T, Z_j)
        
        # Handle potential numerical issues with matrix inversion
        try:
            # Add small regularization to ensure invertibility
            epsilon = 1e-8
            ZTZ_reg = ZTZ + epsilon * np.eye(nv)
            
            # Compute the inverse of Z_j^T * Z_j
            ZTZ_inv = np.linalg.inv(ZTZ_reg)
            
            # Compute w_j according to the formula
            ones = np.ones(nv)
            denominator = np.dot(ones, np.dot(ZTZ_inv, ones))
            
            if np.abs(denominator) < 1e-10:  # Avoid division by zero
                w_j = np.ones(nv) / nv  # Equal weights if denominator is close to zero
            else:
                w_j = np.dot(ZTZ_inv, ones) / denominator
        except np.linalg.LinAlgError:
            # If matrix is still singular, use equal weights
            w_j = np.ones(nv) / nv
        
        # Handle numerical issues: ensure weights are non-negative
        if np.any(w_j < 0):
            # Project to the simplex if there are negative weights
            w_j = project_simplex(w_j)
        
        # Ensure the constraint sum(w_j) = 1 is satisfied exactly
        w_j = w_j / np.sum(w_j)
        
        # Store the weights in the weight matrices
        for v in range(nv):
            w_list[v][:, j] = w_j[v]
            
    return w_list

def make_global_graph(single_view_graph, class_number, default_gamma=1.0):
    
    # init
    W = init_W(single_view_graph)
    A = init_A(single_view_graph, W)
    P = init_P(A, class_number)
    gamma = default_gamma
    
    for j in tqdm(range(5)):
        A = update_A(P, W, single_view_graph)
        L = compute_laplacian(A)
        P = update_P(L, class_number)
        W = compute_W(A, single_view_graph)

        tqdm.write(f"iteration: {j}, L_rank: {np.linalg.matrix_rank(L)}, gamma: {gamma}")
        # print(sum(W[0][:,0]))
        L_rank = np.linalg.matrix_rank(L)
        if L_rank == X.shape[0] - class_number:
            tqdm.write(f"end at {j}th iteration, L's rank is {L_rank}")
            break
        elif L_rank < X.shape[0] - class_number:
            gamma *= 0.9 
        else:
            gamma *= 1.1
        
    return A

In [5]:
global_graph = make_global_graph(single_view_graph, class_number)

  0%|          | 0/5 [11:00<?, ?it/s]

iteration: 0, L_rank: 1999, gamma: 1.0


 20%|██        | 1/5 [22:01<44:23, 665.93s/it]

iteration: 1, L_rank: 1999, gamma: 1.1


 40%|████      | 2/5 [32:57<33:06, 662.32s/it]

...

 60%|██████    | 3/5 [44:04<21:59, 659.82s/it]

iteration: 3, L_rank: 1999, gamma: 1.3310000000000004


 80%|████████  | 4/5 [55:18<11:02, 662.97s/it]

iteration: 4, L_rank: 1999, gamma: 1.4641000000000006


100%|██████████| 5/5 [55:23<00:00, 664.62s/it]


## 1.4 Cluster

In [6]:
import numpy as np
from scipy.sparse.csgraph import connected_components
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

def cluster_from_similarity(similarity_matrix, method='spectral', n_clusters=None):
    # 确保输入是方阵
    assert similarity_matrix.shape[0] == similarity_matrix.shape[1], "相似度矩阵必须是方阵"
    n = similarity_matrix.shape[0]
    threshold = 1/n
    
    if method == 'connected_components':
        # 基于阈值创建邻接矩阵
        adjacency = (similarity_matrix >= threshold).astype(int)
        # 使用连通分量算法进行聚类
        n_components, labels = connected_components(adjacency, directed=False)
        return labels
    
    elif method == 'spectral':
        # 如果没有指定聚类数量，尝试自动估计
        if n_clusters is None:
            # 使用特征值差值估计聚类数量
            laplacian = np.diag(np.sum(similarity_matrix, axis=1)) - similarity_matrix
            eigvals = np.sort(np.linalg.eigvalsh(laplacian))[:10]  # 考虑前10个特征值
            # 计算连续特征值的差值
            gaps = np.diff(eigvals)
            # 选择差值最大的位置作为聚类数量的估计
            if len(gaps) > 0:
                n_clusters = np.argmax(gaps) + 1
                n_clusters = max(2, min(n_clusters, n // 2))  # 限制在合理范围内
            else:
                n_clusters = 2  # 默认为2个聚类
        
        # 构建归一化拉普拉斯矩阵
        D = np.diag(np.sum(similarity_matrix, axis=1))
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.maximum(np.diag(D), 1e-10)))
        L = np.eye(n) - D_inv_sqrt @ similarity_matrix @ D_inv_sqrt
        
        # 计算特征值和特征向量
        eigvals, eigvecs = np.linalg.eigh(L)
        # 选择最小的n_clusters个特征值对应的特征向量
        indices = np.argsort(eigvals)[:n_clusters]
        features = eigvecs[:, indices]
        
        # 对特征向量进行行归一化
        norm = np.sqrt(np.sum(features**2, axis=1)).reshape(-1, 1)
        norm[norm == 0] = 1  # 避免除以零
        features = features / norm
        
        # 使用K-means对特征向量进行聚类
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
        labels = kmeans.fit_predict(features)
        
        return labels
    
    else:
        raise ValueError(f"不支持的聚类方法: {method}，请使用 'connected_components' 或 'spectral'")


# get clustering results
single_view_graph_labels = []
for i in range(len(single_view_graph)):
    single_view_graph_labels.append(cluster_from_similarity(single_view_graph[i], n_clusters = class_number))

global_graph_labels = cluster_from_similarity(global_graph, n_clusters = class_number)

## 1.5 evaluation

In [7]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics.cluster import contingency_matrix

def cluster_accuracy(y_true, y_pred):
    acc = np.mean(y_pred == y_true)
    return acc

def purity_score(y_true, y_pred):
    contingency = contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency, axis=0)) / np.sum(contingency)

def pairwise_precision_recall_fscore(y_true, y_pred):

    def get_pairs(labels):
        pairs = set()
        for label in np.unique(labels):
            indices = np.where(labels == label)[0]
            for i in range(len(indices)):
                for j in range(i + 1, len(indices)):
                    pairs.add((indices[i], indices[j]))
        return pairs

    true_pairs = get_pairs(y_true)
    pred_pairs = get_pairs(y_pred)
    
    tp = len(true_pairs & pred_pairs)
    fp = len(pred_pairs - true_pairs)
    fn = len(true_pairs - pred_pairs)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    
    return precision, recall, f_score

def evaluate_clustering(y_true, y_pred):
    
    # remapping 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    assert y_true.shape == y_pred.shape

    labels = np.unique(y_true)
    pred_labels = np.unique(y_pred)
    cost_matrix = -contingency_matrix(y_true, y_pred)

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    best_mapping = {pred_labels[col]: labels[row] for row, col in zip(row_ind, col_ind)}

    y_pred_mapped = np.array([best_mapping[label] for label in y_pred])

    # evaluate
    acc = cluster_accuracy(y_true, y_pred_mapped)
    nmi = normalized_mutual_info_score(y_true, y_pred)
    purity = purity_score(y_true, y_pred_mapped)
    precision, recall, f_score = pairwise_precision_recall_fscore(y_true, y_pred_mapped)
    ari = adjusted_rand_score(y_true, y_pred_mapped)

    return {
        "ACC": acc,
        "NMI": nmi,
        "Purity": purity,
        "Precision": precision,
        "Recall": recall,
        "F-score": f_score,
        "ARI": ari
    }


In [9]:
metrics = evaluate_clustering(labels, global_graph_labels)
print(metrics)


{'ACC': 0.148, 'NMI': 0.049214620507958426, 'Purity': 0.163, 'Precision': 0.10451862458951827, 'Recall': 0.40272864321608043, 'F-score': 0.1659649900288057, 'ARI': 0.009371416335532866}
